# Exemplo 03a: Análise de Enlace usando NetworkX
## Identificação de Aeroportos com mais conectividade

Muitos problemas em ciência de dados podem ser modelados com um grafo. Um exemplo é a análise da rede aérea em uma região, onde os vertices são os aeroportos e as areastas são as linhas aéreas. Usando algoritmos de análise de enlace podemos extrair informações como aeroportos mais movimentados e menor caminho entre duas localidades.

Porém essa análise não é tão trivial. A maneira mais simples de determinar o aeroporto mais movimentado é contar o número de voos realizados de e para esta cidade. No entanto, como a maioria das companhias aéreas utiliza um sistema de *hub-and-spoke*, a simples contagem dos voos de entrada e saída não transmite a importância do aeroporto para o tráfego aéreo geral. Isso ocorre porque determinados aeroportos centrais podem ser pontos de passagem para os vôos em outros aeroportos e, como resultado, esses aeroportos centrais podem ser considerados mais importantes, mesmo que tenham contagens total de voos seja igual ou até menores.

O algoritmo Pagerank foi originalmente criado para medir a importância relativa das páginas da web, avaliando os links de ligação da página. Qualquer página da web é considerada mais importante se outras páginas importantes tiverem links para essa página. Podemos aplicar esse mesmo conceito de importância aos aeroportos. Se você substituir “página da web” por “aeroporto” e substituir “link da web” por “voo da companhia aérea”, poderá ver que o PageRank pode ser usado para avaliar a importância de um aeroporto. O PageRank acaba sendo uma boa maneira de medir a importância do aeroporto, dado o uso do molelo *hub-and-spoke* usado pelas companhias hub e spoke das companhias aéreas. Um aeroporto importante acabaria sendo um aeroporto que por si só é um *hub* no qual outros aeroportos possuem muitos vôos ou um “hub de hubs”.

Para este exemplo vamos usar uma base de dados de aeroportos e voos nos Estados Unidos e Canada. Então, para identificar os aeroportos mais importantes da região, considerando como um determinado aeroporto influencia os vôos para outros aeroportos, podemos usar o algoritmo Pagerank. Este exemplo mostra os aeroportos com mais voos e os aeroportos mais conectados calculados pelo Pagerank.

In [1]:
import pyspark
from pyspark.sql import SparkSession

import networkx as nx

import time
start_time = time.time()

data_path='./data/'

In [2]:
# Cria Spark Session
sc = SparkSession.builder \
     .master("local[*]") \
     .appName("LinkAnalisysAirlines") \
     .getOrCreate()

## Reading datasets
### Airports

| Field | Description |
| --------- | :-------------: |
| node_id | Unique identifier for the airport. |
| name | Name of airport or city and state.|
| metro_pop | City/region population.|
| latitude | Airport latitude |
| longitude | Airport longitude |


In [3]:
# Le arquivo de dados Airport para Dataframe Spark

airport = sc.read.format("csv").options(sep=',',header='true',inferschema='true').\
          load(data_path+"reachability-meta.csv.gz")

#Exibe campos da tabela e os tipos de dados
#airport.dtypes
airport.show()

+-------+--------------------+---------+---------+-----------+
|node_id|                name|metro_pop| latitude|  longitude|
+-------+--------------------+---------+---------+-----------+
|      0|      Abbotsford, BC| 133497.0|49.051575|-122.328849|
|      1|        Aberdeen, SD|  40878.0| 45.45909| -98.487324|
|      2|         Abilene, TX| 166416.0|32.449175| -99.741424|
|      3|    Akron/Canton, OH| 701456.0| 40.79781| -81.371567|
|      4|         Alamosa, CO|   9433.0| 37.46818|-105.873599|
|      5|          Albany, GA| 157688.0| 31.58076| -84.155989|
|      6|          Albany, NY| 871478.0|42.651455| -73.755274|
|      7|     Albuquerque, NM| 898642.0| 35.08418|-106.648639|
|      8|      Alexandria, LA| 154505.0|31.312685| -92.445649|
|      9|Allentown/Bethleh...| 824916.0|40.651428| -75.434219|
|     10|        Alliance, NE|   8499.0| 42.09712|-102.871454|
|     11|          Alpena, MI|  29386.0|45.061565| -83.445154|
|     12|         Altoona, PA| 127099.0| 40.50719| -78.

### Flights

| Field | Description |
| --------- | :-------------: |
| FromNodeId | Origin airport (node_id).|
| ToNodeId | Destination airport (node_id).|
| Weight | Distance |


In [4]:
# Le arquivo de dados Linhas Aereas para Dataframe Spark

routes = sc.read.format("csv").options(sep=' ',header='true',inferschema='true').\
         load(data_path+"reachability.txt.gz")

#routes.dtypes
routes.show()

+----------+--------+------+
|FromNodeId|ToNodeId|Weight|
+----------+--------+------+
|        27|       0|  -757|
|        57|       0|   -84|
|        70|       0| -1290|
|        74|       0|  -465|
|        86|       0|  -700|
|        94|       0|  -526|
|       100|       0|  -448|
|       113|       0|   -90|
|       138|       0|  -256|
|       154|       0|  -270|
|       166|       0|  -515|
|       178|       0|  -400|
|       230|       0|  -486|
|       235|       0|  -170|
|       242|       0|  -469|
|       246|       0|  -325|
|       262|       0|  -200|
|       269|       0|  -525|
|       275|       0|  -585|
|       280|       0|  -405|
+----------+--------+------+
only showing top 20 rows



## Building Graph

In [5]:
# Extrair campos relevantes para definir os vertices do grafo
vertice = airport.select("node_id","name")

# Troca nome "node_id" por "id"
vertice = vertice.withColumnRenamed("node_id", "id")

#caso precise converter algum campo
#vertice = vertice.withColumn("id", vertice["id"].cast("string"))

# Converte Spark Dataframe para Pandas Dataframe
vertice_pd = vertice.toPandas()

# Salva lista com nomes das cidades ccom aeroportos
airport_name = airport.select("node_id", "name").withColumnRenamed("node_id", "id")

#vertice.dtypes
vertice.show(5)

+---+----------------+
| id|            name|
+---+----------------+
|  0|  Abbotsford, BC|
|  1|    Aberdeen, SD|
|  2|     Abilene, TX|
|  3|Akron/Canton, OH|
|  4|     Alamosa, CO|
+---+----------------+
only showing top 5 rows



In [6]:
# Extrair campos relevantes para definir os arestas do grafo
edge = routes.select("FromNodeId","ToNodeId","Weight")

# Graphframe exige que colunas com identificação das arestas possua nome 'src' para origem e 'dst' para destino
# Troca nome "FromNodeId" por "src" e "ToNodeId" por "dst"
edge = edge.withColumnRenamed("FromNodeId", "src") \
           .withColumnRenamed("ToNodeId", "dst") \
           .withColumnRenamed("Weight", "weight")

# Converte Spark Dataframe para Pandas Dataframe
edge_pd = edge.toPandas()

edge.show(5)

+---+---+------+
|src|dst|weight|
+---+---+------+
| 27|  0|  -757|
| 57|  0|   -84|
| 70|  0| -1290|
| 74|  0|  -465|
| 86|  0|  -700|
+---+---+------+
only showing top 5 rows



In [7]:
# Cria um Digraph com as rotas das linhas aéreas 
# edge_pd é o gdataframe com as linhas aereas  no formato Pandas
# sorce e target ao aeroportos de origem e destino das linhas aereas
# edge_attr é o atributo da aresta do grafo, neste caso o valor negativo da distância (quanto mais distante pior) 

G = nx.from_pandas_edgelist(edge_pd, source='src', target='dst', edge_attr=['weight'], create_using=nx.DiGraph())

print(G)

DiGraph with 456 nodes and 71959 edges


In [8]:
# imprime caracteristica do grafo

G_nodes = G.number_of_nodes()
G_edges = G.number_of_edges()

print("Airports = ", G_nodes, "      Flights = ",G_edges)

Airports =  456       Flights =  71959


## In-degree of each vertex in the graph

In [9]:
# grau de entrada: Identifica aeroportos com mais chegadas
airport_degree = nx.degree(G)

# Define columns name
label = ["id", "degree"] 
  
# creating a Spark dataframe from result degree
airport_degree_df = sc.createDataFrame(airport_degree, label)

airport_degree_df.select("id", "degree").join(airport_name, on=['id'], how='inner').select("name", "degree").orderBy("degree",ascending=False).show()

+--------------------+------+
|                name|degree|
+--------------------+------+
|     Los Angeles, CA|   886|
|   San Francisco, CA|   871|
|       Las Vegas, NV|   861|
|         Chicago, IL|   859|
|Dallas/Fort Worth...|   859|
|        New York, NY|   856|
|          Denver, CO|   856|
|      Washington, DC|   852|
|         Phoenix, AZ|   847|
|  Seattle/Tacoma, WA|   824|
|Minneapolis/St Pa...|   817|
|         Toronto, ON|   813|
|         Orlando, FL|   805|
|    Philadelphia, PA|   805|
|          Boston, MA|   803|
|  Ft. Lauderdale, FL|   801|
|         Houston, TX|   800|
|       San Diego, CA|   799|
|         Detroit, MI|   786|
|           Miami, FL|   786|
+--------------------+------+
only showing top 20 rows



## Triangle Count for each vertice

In [10]:
# Count the number of triagles in each vertice: aeroportos com mais chegadas incluindo uma escala
triang = G.in_degree

# Define columns name
label = ["id", "count"] 
  
# creating a Spark dataframe
triang_df = sc.createDataFrame(triang, label)

triang_df.select("id", "count").join(airport_name, on=['id'], how='inner').select("name", "count").orderBy("count",ascending=False).show()

+--------------------+-----+
|                name|count|
+--------------------+-----+
|     Los Angeles, CA|  443|
|   San Francisco, CA|  441|
|Dallas/Fort Worth...|  432|
|         Chicago, IL|  429|
|       Las Vegas, NV|  429|
|        New York, NY|  428|
|          Denver, CO|  427|
|      Washington, DC|  425|
|         Phoenix, AZ|  423|
|  Seattle/Tacoma, WA|  418|
|Minneapolis/St Pa...|  411|
|         Orlando, FL|  407|
|         Toronto, ON|  404|
|    Philadelphia, PA|  403|
|       St. Louis, MO|  401|
|         Houston, TX|  400|
|  Ft. Lauderdale, FL|  400|
|          Boston, MA|  399|
|       San Diego, CA|  397|
|         Detroit, MI|  393|
+--------------------+-----+
only showing top 20 rows



## Page Rank

In [11]:
# Run PageRank algorithm, and show results: aeroportos mais conectados

result_pr = nx.pagerank(G, max_iter=20, weight='weight')
#result_pr = nx.pagerank(G, tol=0.001, weight='weight')

# Convert dict to list
result_pr_l = list(result_pr.items())

# Define columns name
label = ["id", "pagerank"] 
  
# creating a Spark dataframe
result_pr_df = sc.createDataFrame(result_pr_l, label)
                      
#result_pr_df.printSchema()
#result_pr_df.show(truncate=False)

result_pr_df.select("id", "pagerank").join(airport_name, on=['id'], how='inner').select("name", "pagerank").orderBy("pagerank",ascending=False).show()

+--------------------+--------------------+
|                name|            pagerank|
+--------------------+--------------------+
|        Honolulu, HI| 0.00818388309618344|
|         Kahului, HI|0.007168021529069031|
|   San Francisco, CA|0.006595916928486906|
|       Anchorage, AK|0.006504644905028423|
|     Los Angeles, CA|0.006186437087956...|
|  Ft. Lauderdale, FL|0.005834928842374449|
|       Las Vegas, NV|0.005764698625665773|
|  Seattle/Tacoma, WA|0.005762596900681118|
|        Portland, OR|0.005736525079205345|
|       San Diego, CA|0.005717626667160882|
|            Kona, HI|0.005649699077920976|
|         Orlando, FL|0.005412397245067099|
|        New York, NY|0.005392278484662518|
|           Miami, FL|0.005383153438456302|
|         Phoenix, AZ|0.005341121969895439|
|       Vancouver, BC|0.005124175450836...|
|Dallas/Fort Worth...|0.005106744140501227|
|      Washington, DC|0.005018072381580718|
|        Hartford, CT|0.004995100897124851|
|     New Orleans, LA|0.00494961

## HITS

In [12]:
# Run HITS algorithm, and show results.

result_h, result_a = nx.hits(G, max_iter=20, normalized=True)
#result_h, result_a = nx.hits(G, tol=0.001, normalized=True)

# Show Hub Vector

# Convert dict to list
result_h_l = list(result_h.items())

# Define columns name
label = ["id", "hub"] 

# creating a Spark dataframe
result_h_df = sc.createDataFrame(result_h_l, label)

#result_h_df.printSchema()
#result_h_df.show(truncate=False)

result_h_df.select("id", "hub").join(airport_name, on=['id'], how='inner').select("name", "hub").orderBy("hub",ascending=False).show()

+--------------------+--------------------+
|                name|                 hub|
+--------------------+--------------------+
|        Honolulu, HI|0.007384033086297942|
|       Anchorage, AK|0.006088494735881721|
|            Kona, HI|0.005916491000365931|
|         Kahului, HI|0.005838290030427...|
|  Ft. Lauderdale, FL|0.005155219326702114|
|           Miami, FL|0.004792663104053244|
|         Halifax, NS|0.004704886799526...|
|       Vancouver, BC|0.004645868643909...|
|   Orange County, CA| 0.00464360273079791|
|Tampa/St. Petersb...|0.004512377162523707|
|   San Francisco, CA|0.004509636808009571|
|        San Jose, CA|0.004475402340925312|
|        Portland, OR|0.004434156495931232|
|       San Diego, CA|0.004371634458802095|
|         Orlando, FL|0.004369440036767276|
|          Boston, MA|0.004356415019147119|
|     Los Angeles, CA|0.004289385081402662|
|    Jacksonville, FL|0.004271544383068075|
|           Boise, ID|0.004245247370999...|
|      Providence, RI|0.00423378

In [13]:
# Show Authority Vector

# Convert dict to list
result_a_l = list(result_a.items())

# Define columns name
label = ["id", "authority"] 

# creating a Spark dataframe
result_a_df = sc.createDataFrame(result_a_l, label)

result_a_df.select("id", "authority").join(airport_name, on=['id'], how='inner').select("name", "authority").orderBy("authority",ascending=False).show()


+------------------+--------------------+
|              name|           authority|
+------------------+--------------------+
|      Honolulu, HI|0.007761193463890724|
|       Kahului, HI|0.007217679977156241|
|     Anchorage, AK|0.006502391625816071|
|          Kona, HI|0.006212735692198184|
|     Fairbanks, AK|0.005921170245835963|
| San Francisco, CA|0.004710982632961002|
|      Portland, OR|0.004587041039913923|
|     Vancouver, BC|0.004543316093029993|
|   Los Angeles, CA|0.004475430805002405|
|Ft. Lauderdale, FL|0.004459072523804407|
|Seattle/Tacoma, WA|0.004441640263958...|
|      Edmonton, AB| 0.00441896506493055|
|     San Diego, CA|0.004398726551017628|
|    Sacramento, CA|0.004380877699646041|
|       Ontario, CA|0.004331599754886468|
|      San Jose, CA|0.004327541966068726|
|       Burbank, CA|0.004294573571350849|
|         Lihue, HI|0.004263380862235175|
|         Miami, FL|0.004256567335794641|
|       Calgary, AB| 0.00418276070908016|
+------------------+--------------

In [14]:
sc.stop()
print("--- Execution time: %s seconds ---" % (time.time() - start_time))

--- Execution time: 17.529409885406494 seconds ---
